# TOC

__Chapter 3 - Understanding TensorFlow Basics__

1. [Import](#Import)
1. [Graphs, sessions, and fetches](#Graphs-Sessions-and-Fetches)
    1. [Creating a graph](#Creating-a-graph)
    1. [Creating a session and running it](02#Creating-a-Session-and-Running-It)
    1. [Constructing and managing our graph](#Constructing-and-Managing-Our-Graph)
    1. [Fetches](#Fetches)
1. [Flowing tensors](#Flowing-Tensors)
    1. [Tensor arrays and shapes](#Tensor-Arrays-and-Shapes)
    1. [Matrix multiplication](#Matrix-multiplication)
    1. [Names](#names)
    1. [Name scopes](#Name-scopes)
1. [Variables, placeholders, and simple optimization](#Variables-placeholders-and-simple-optimization)
    1. [Variables](#Variables)
    1. [Placeholders](#Placeholders)
    1. [Optimization](#Optimization)
        1. [Linear regression](#Linear-regression)
        1. [Logistic regression](#Logistic-regression)

<a id = 'Import'></a>

# Import

In [1]:
# Standard libary and settings
import os
import sys
import importlib
import itertools
import warnings; warnings.simplefilter('ignore')
modulePath = os.path.abspath(os.path.join('../../CustomModules'))
sys.path.append(modulePath) if modulePath not in sys.path else None
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:95% !important; }</style>"))


# Data extensions and settings
import numpy as np
np.set_printoptions(threshold = np.inf, suppress = True)
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,.6f}'.format

import tensorflow as tf

# Visualization extensions and settings
import seaborn as sns
import matplotlib.pyplot as plt


# Custom extensions and settings
import mlmachine as mlm


# Magic functions
%matplotlib inline


<a id = 'Graphs-Sessions-and-Fetches'></a>

# Graphs, sessions, and fetches

<a id = 'Creating-a-graph'></a>

## Creating a graph

In [2]:
# Create six nodes, which will be automatically associated
# with the default graph

# define 3 constants
a = tf.constant(5)
b = tf.constant(2)
c = tf.constant(3)

# define 2 nodes that perform simple arithmetic operations
# the constants defined above

d = tf.multiply(a,b)
e = tf.add(c,b)

# define 1 last node that performs arithmetic operations
# on the two operation nodes above
f = tf.subtract(d,e)


> Remarks - In graph form, 
- c & b are connected to e
- b & a are connected to d
- e & d are connected to f

<a id = 'Creating-a-Session-and-Running-It'></a>

## Creating a session and running It

In [3]:
# Create a session and run, then print output

sess = tf.Session()
outs = sess.run(f)
sess.close()

print('outs = {}'.format(outs))


outs = 5


<a id = 'Constructing-and-Managing-Our-Graph'></a>

## Constructing and managing our graph

In addition to the default graph that is automatically created upon import of TensorFlow, we can create additional graphs.

In [4]:
# Print default graph and create new empty graph

print(tf.get_default_graph())

g = tf.Graph()
print(g)


In [5]:
# Display the graph which constant 'a' is associated with
# by using 'a.graph'

g = tf.Graph()
a = tf.constant(5)

print(a.graph is g)
print(a.graph is tf.get_default_graph())


False
True


In [6]:
# To associate nodes with a new graph, use the 'with' statement
# Note - a session doesn't need to be closed when using 'with'
g1 = tf.get_default_graph()
g2 = tf.Graph()

print(g1 is tf.get_default_graph())

with g2.as_default():
    print(g1 is tf.get_default_graph())
    
print(g1 is tf.get_default_graph())


True
False
True


<a id = 'Fetches'></a>

## Fetches

In section 3.2, we passed the node 'f' as an argument to sess.run(), which ran all essential nodes needed to complete the operation of 'f'. This argument used to complete the request to 'f' is called 'fetches', which communicate the elements of the graph we want to compute.

We can also ask sess.run() to evaluate multiple nodes.

In [7]:
#

with tf.Session() as sess:
    fetches = [a,b,c,d,e]
    outs = sess.run(fetches)
    
print('outs = {}'.format(outs))
print(type(outs[0]))


outs = [5, 2, 3, 10, 5]
<class 'numpy.int32'>


<a id = 'Flowing-Tensors'></a>

# Flowing tensors

Data types and shapes of objects in TensorFlow are automatically selected by the TensorFlow API but can be explicitly declared as needed.

<a id = 'Tensor-Arrays-and-Shapes'></a>

## Tensor arrays and shapes

In [8]:
#

c = tf.constant([[1,2,3]
                ,[4,5,6]])
print('Python list input: {}'.format(c.get_shape()))

c = tf.constant(np.array([
                [[1,2,3,4]
                ,[5,6,7,8]
                ,[9,8,7,6]]
                
               ,[[1,1,1,1]
                ,[2,2,2,2]
                ,[3,3,3,3]]
            ]))
print('3d NumPy array input: {}'.format(c.get_shape()))


Python list input: (2, 3)
3d NumPy array input: (2, 3, 4)


<a id = 'Matrix multiplication'></a>

## Matrix multiplication

tf.matmul(A,B) is an operator for performing matrix multiplicaiton between two TensorFlow objects A and B.

In [9]:
# Create and print basic matrices

A = tf.constant([ [1,2,3]
                  ,[4,5,6] ])
print(A.get_shape())

x = tf.constant([1,0,1])
print(x.get_shape())


(2, 3)
(3,)


In [10]:
# Transform 'x' into a 2D matrix to allow matrix multiplication

x = tf.expand_dims(x,1)
print(x.get_shape())

b = tf.matmul(A,x)

sess = tf.InteractiveSession()
print('matmul result: \n {}'.format(b.eval()))
sess.close()


(3, 1)
matmul result: 
 [[ 4]
 [10]]


<a id = 'names'></a>

## Names

The name of a Tensor object is the name of it's corresponding operation (below, "c") concatenated with a semi-colon, followed by the index of that tensor in the ouputs of the operation that produced it.

In [11]:
#

with tf.Graph().as_default():
    c1 = tf.constant(4, dtype = tf.float64, name = 'c')
    c2 = tf.constant(4, dtype = tf.int32, name = 'c')
print(c1.name)
print(c2.name)


c:0
c_1:0


<a id = 'Name-scopes'></a>

## Name scopes

In a large complicated graph, it can be helpful to create node groupings to make it easier to follow and manage the graph. Nodes can be grouped together by name.

In [12]:
#

with tf.Graph().as_default():
    c1 = tf.constant(4, dtype = tf.float64, name = 'c')
    with tf.name_scope('my_prefix'):
        c2 = tf.constant(4, dtype = tf.int32, name = "c")
        c3 = tf.constant(4, dtype = tf.float64, name = "c")

print(c1.name)
print(c2.name)
print(c3.name)

        

c:0
my_prefix/c:0
my_prefix/c_1:0


<a id = 'Variables-placeholders-and-simple-optimization'></a>

# Variables placeholders and simple optimization

<a id = 'Variables'></a>

## Variables

In [18]:
# Demonstrating the need to initializer variables

init_val = tf.random_normal((1,5),0,1)
var = tf.Variable(init_val, name = 'var')
print('pre run: \n {}'.format(var))

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    post_var = sess.run(var)
    
print('post run: \n{}'.format(post_var))

pre run: 
 <tf.Variable 'var_3:0' shape=(1, 5) dtype=float32_ref>
post run: 
[[-1.414155   -0.8080999   1.876038    0.08628396  1.4195226 ]]


<a id = 'Placeholders'></a>

## Placeholders

Placeholders are structures for feeding input values. They can be thought of as empty Variables that will be filled with data once the graph is executed.

Placeholders have an option shape argument. If nothing passed, or None is passed, the placeholder can be fed with data of any size. None is commonly used for the dimension of a matrix that corresponds to the number of sample/rows, as this will vary, while the column are more commonly fixed.

Data is passed to the placeholder via a dictionary, where each key corresponds to a placeholder variable name. The corresponding values are given in the form of a list or a numpy array

In [28]:
# 

X_data = np.random.randn(5,10)
w_data = np.random.randn(10,1)

with tf.Graph().as_default():
    x = tf.placeholder(tf.float32, shape = (5,10))
    w = tf.placeholder(tf.float32, shape = (10,1))    
    b = tf.fill((5,1),-1.)
    xw = tf.matmul(x,w)
    
    xwb = xw + b
    s = tf.reduce_max(xwb)
    
    with tf.Session() as sess:
        outs = sess.run(s, feed_dict = {x : X_data, w : w_data})
    
    print('outs = {}'.format(outs))
    

Tensor("add:0", shape=(5, 1), dtype=float32)
outs = 2.684581756591797


<a id = 'Optimization'></a>

## Optimization

Optimization illustrated with simple linear regression: $f(x_i) = \hat{y}_i = w^tx_i + b$, where $y_i = f(x_i) + \epsilon_i$ and we want to minimize the mean squared error.


<a id = 'Linear-regression'></a>

### Linear regression

In [49]:
# define placeholders and variables

X_data = np.random.randn(2000,3)
w_real = [0.3, 0.5, 0.1]
b_real = -0.2

noise = np.random.randn(1,2000) * 0.1
y_data = np.matmul(w_real, X_data.T) + b_real + noise

steps = 10

g = tf.Graph()
wb_ = []

with g.as_default():
    x = tf.placeholder(tf.float32, shape = [None,3])
    y_true = tf.placeholder(tf.float32, shape = None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]], dtype = tf.float32, name = 'weights')
        b = tf.Variable(0, dtype = tf.float32, name = 'bias')
        y_pred = tf.matmul(w, tf.transpose(x)) + b
        
    with tf.name_scope('loss') as scope:
        loss = tf.reduce_mean(tf.square(y_true - y_pred))
        
    with tf.name_scope('train') as scope:
        learning_rate = 0.5
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train = optimizer.minimize(loss)
        
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for step in range(steps):
            sess.run(train, feed_dict = {x : X_data, y_true : y_data})
            if (step % 2 == 0):
                print(step, sess.run([w,b]))
                wb_.append(sess.run([w,b]))
        print(10, sess.run([w,b]))
        

0 [array([[0.28120708, 0.5166322 , 0.10044156]], dtype=float32), -0.18235394]
2 [array([[0.2977588 , 0.49611494, 0.10199146]], dtype=float32), -0.19899043]
4 [array([[0.29778978, 0.49600846, 0.10197536]], dtype=float32), -0.19904207]
6 [array([[0.29778987, 0.49600795, 0.10197516]], dtype=float32), -0.19904228]
8 [array([[0.29778987, 0.49600795, 0.10197516]], dtype=float32), -0.19904228]
10 [array([[0.29778987, 0.49600795, 0.10197515]], dtype=float32), -0.19904229]


> Remarks - The weight are very close to the 'w_real' weights defined at the top of the cell

<a id = 'Logistic-regression'></a>

### Logistic regression

In [56]:
#

n = 20000
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

X_data = np.random.randn(n, 3)
w_real = [0.3, 0.5, 0.1]
b = -0.2
wxb = np.matmul(w_real, X_data.T) + b

y_data_pre_noise = sigmoid(wxb)
y_data = np.random.binomial(1, y_data_pre_noise)

# y_pred = tf.sigmoid(y_pred)
# loss = -y_true * tf.log(y_pred) - (1 - y_true) * tf.log(1 - y_pred)
# loss = tf.reduce_mean(loss)

steps = 50

with g.as_default():
    x = tf.placeholder(tf.float32, shape = [None,3])
    y_true = tf.placeholder(tf.float32, shape = None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]], dtype = tf.float32, name = 'weights')
        b = tf.Variable(0, dtype = tf.float32, name = 'bias')
        y_pred = tf.matmul(w, tf.transpose(x)) + b
        
    with tf.name_scope('loss') as scope:
        loss = tf.nn.sigmoid_cross_entropy_with_logits(labels = y_true, logits = y_pred)
        loss = tf.reduce_mean(loss)
        
    with tf.name_scope('train') as scope:
        learning_rate = 0.5
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train = optimizer.minimize(loss)
    
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for step in range(steps):
            sess.run(train, feed_dict = {x : X_data, y_true : y_data})
            if (step % 5 == 0):
                print(step, sess.run([w,b]))
                wb_.append(sess.run([w,b]))
        print(50, sess.run([w,b]))



0 [array([[0.03334042, 0.05834084, 0.01118399]], dtype=float32), -0.021324975]
5 [array([[0.14800419, 0.25919873, 0.04945645]], dtype=float32), -0.09417923]
10 [array([[0.20966256, 0.3674317 , 0.06989127]], dtype=float32), -0.13284999]
15 [array([[0.24427655, 0.42832217, 0.08131289]], dtype=float32), -0.1542685]
20 [array([[0.26427403, 0.4635718 , 0.08789396]], dtype=float32), -0.16648132]
25 [array([[0.2760289 , 0.48433077, 0.09175595]], dtype=float32), -0.1735723]
30 [array([[0.28301036, 0.4966806 , 0.09404707]], dtype=float32), -0.17773612]
35 [array([[0.28718242, 0.50407183, 0.0954151 ]], dtype=float32), -0.18019867]
40 [array([[0.28968486, 0.5085111 , 0.09623513]], dtype=float32), -0.18166184]
45 [array([[0.29118922, 0.511183  , 0.0967278 ]], dtype=float32), -0.18253393]
50 [array([[0.29194844, 0.5125328 , 0.09697632]], dtype=float32), -0.18297094]
